In [1]:
import pandas as pd
# import glob
# import imageio
# import matplotlib.pyplot as plt
import numpy as np
import os
# import PIL
from tensorflow import keras
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input
from keras.models import Model, model_from_json
from keras.layers.normalization import BatchNormalization
import time
# from IPython import display
import math
import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# import tensorflow as tf
# import json
# from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
def minMaxNormalise(data, cont_ls):
    for j in cont_ls:
        index = list(data.index.values)
        df_ = pd.DataFrame(index=index, columns=[j])
        for i in index:
            df_.at[i,j] = (data.at[i,j]-data[j].min())/(data[j].max()-data[j].min())
        data.at[:,j] = np.nan
        for k in index:
            data.at[k,j] = df_.at[k,j]
    
    return data

In [3]:
def meanNormalise(data, cont_ls):
    for j in cont_ls:
        index = list(data.index.values)
        df_ = pd.DataFrame(index=index, columns=[j])
        for i in index:
            df_.at[i,j] = (data.at[i,j]-data[j].mean())/(data[j].std())
        data.at[:,j] = np.nan
        for k in index:
            data.at[k,j] = df_.at[k,j]
    
    return data

In [4]:
#one hot encode categorical variable columns
def oneHotEnc(data, cat_ls):
    for i in cat_ls:
        col_df = data[i]
        encoded_df = pd.get_dummies(col_df, prefix = [i])
        data = data.drop([i], axis=1)
        data = pd.concat([data, encoded_df], axis = 1, sort=False)
    return data

In [5]:
#generate random input for GAN generator
def sample_z(m, n):
    seed = np.random.uniform(0., 1., size=[m, n])
    seed = np.sort(seed)
    return seed

In [6]:
def contToCat(data, num_bins, columns):
    data = data
    for i in columns:
        bin_sz = math.floor((data[i].max() - data[i].min())/num_bins)
        bins = []
        for a in range(0, num_bins+1):
            bins.append(data[i].min() + a*bin_sz)
        data[i] = pd.cut(x=data[i], bins = bins)
    data = oneHotEnc(data, columns)
    return data

In [ ]:
tf.keras.backend.clear_session
tf.reset_default_graph()
start = time.time()
data = pd.read_csv('heart-disease-uci.zip')
cat_col = ['cp', 'slope', 'thal', 'target'] #columns in dataframe which contain categorical variables
# data_cat = data[cat_col] #Select columns in dataframe which contain categorical variables
data_cat = oneHotEnc(data, cat_col) 
num_bins = 5
cont_ls = ['age', 'trestbps', 'chol', 'thalach','oldpeak']
data_cat_2 = contToCat(data_cat, num_bins, cont_ls)
cohort_size = 50 #size of the cohort which we want GAN to simulate (i.e. 50 patients)
data_train, data_test, _, _ = train_test_split(data_cat_2, data_cat_2, test_size=0.33) #split dataset into train and test, test will only be used to test accuracy of GAN generated patients
data_train_list = []
data_test_list = []
for i in range(300): #Here we are creating 300 training and test samples of 50 patients each 
    data_train_list.append(data_train.sample(cohort_size).values.flatten())        
    data_test_list.append(data_test.sample(cohort_size).values.flatten())

#Reshape train and test lists into arrays of 300 x cohort size*number of features per patient
data_train_cohort = np.reshape(data_train_list, (300, cohort_size*data_cat_2.shape[1]))
data_test_cohort = np.reshape(data_test_list, (300, cohort_size*data_cat_2.shape[1]))

data_train_cohort = pd.DataFrame(data_train_cohort)
print(data_cat_2.shape[1])
#Autoencoder code
num_filters = 200
kernel_size = 100
strides = 5
encoding_dim = 50000
encoded_input_sz = (((cohort_size*data_cat_2.shape[1] - kernel_size)/strides)+1)*num_filters
input_ = Input(shape=(cohort_size*data_cat_2.shape[1],), name = "Input")
input__ = Reshape((cohort_size*data_cat_2.shape[1],1))(input_)
# conv_1D = Conv1D(filters = num_filters, kernel_size = kernel_size, strides = strides, input_shape = (cohort_size*data_cat_2.shape[1],1), name = "conv_layer")
encoded = Dense(encoding_dim, activation='relu')(input_)
# encoded = conv_1D(input__)
# encoded_1 = Flatten()(encoded) #Only for conv1D
decoding_layer = Dense(cohort_size*data_cat_2.shape[1], activation = 'sigmoid', name = "layer_4")
# decoded = decoding_layer(encoded_1) #Only for Conv1D
decoded = decoding_layer(encoded)

autoencoder = Model(input_, decoded)
encoder = Model(input_, encoded)

# encoded_input = Input(shape=(encoded_input_sz,))
encoded_input = Input(shape=(encoding_dim,))
decoder_layer_3 = autoencoder.layers[-1](encoded_input)
decoder = Model(encoded_input, decoder_layer_3)

autoencoder.compile(optimizer='Adam', loss='binary_crossentropy')
autoencoder.fit(data_train_cohort, data_train_cohort, epochs=500, validation_data=(data_test_cohort, data_test_cohort), verbose=False)
print((time.time()-start)/60)
# print(data_test.iloc[1].values.reshape(data_test.shape[1],1).shape)
# print(data_test)
# decoder.predict(encoder.predict(data_test)).round(0)
# print(decoder.predict(encoder.predict(data_test.iloc[1].values.reshape(data_test.shape[1]))))

# print(autoencoder.layers[-1].get_weights()[0])
##############################################################
autoencoded_data = decoder.predict(encoder.predict(data_test_cohort)).round(0)
accuracy_list = []
_0 = 0
_1 = 0
_2 = 0
_3 = 0
for i in range(autoencoded_data.shape[0]):
    total = 0
    for a in range(cohort_size):
        predicted = np.reshape(autoencoded_data[i], (cohort_size, data_cat_2.shape[1]))[a]
        test = np.reshape(data_test_cohort[i], (cohort_size, data_cat_2.shape[1]))[a]
        x = np.sum(abs(predicted - test))
        if x == 0:
            total = total + 1
            _0 = _0 + 1
        if x == 1:
            _1 = _1 + 1
        if x == 2:
            _2 = _2 + 1
        if x >= 3:
            _3 = _3 + 1
    accuracy_list.append(total/cohort_size)
print(np.mean(accuracy_list))
print(autoencoded_data.shape)
print(data_test.shape)
total = 0
for i in range(data_test_cohort.shape[0]):
    for a in range(data_test_cohort.shape[1]):
        if data_test_cohort[i,a] == autoencoded_data[i,a]:
            total = total+1
print("Accuracy % per variable: ", total/(data_test_cohort.shape[0]*data_test.shape[1]))
print("0 errors", _0/(cohort_size*data_test_cohort.shape[0]))
print("1 error", _1/(cohort_size*data_test_cohort.shape[0]))
print("2 errors", _2/(cohort_size*data_test_cohort.shape[0]))
print("3 errors", _3/(cohort_size*data_test_cohort.shape[0]))

# Can I save the autoencoder and reload it later so I don't have to train it everytime? 

43
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
tf.keras.backend.clear_session
tf.reset_default_graph()
def my_innit(shape, dtype=None): #Get weights from decoding layer of autoencoder
    new_autoencoder = keras.Model.from_config(config)
    new_autoencoder.set_weights(weights)
    a = new_autoencoder.get_weights()[-2]
    return a

def my_binnit(shape, dtype=None): #Get biases from decoding layer of autoencoder
    new_autoencoder = keras.Model.from_config(config)
    new_autoencoder.set_weights(weights)
    b = new_autoencoder.get_weights()[-1]
    return b

# num_points = encoding_dim
# print(num_points)
gen_input_size = 15000 #Input size to generator
optimizer = Adam(0.0002, 0.9)
batch_size = 100
epochs = 500
steps_per_epoch = 1
def generator():
    generator = Sequential()
        
#     generator.add(Dense(5000, input_dim=gen_input_size))
#     generator.add(Activation('relu'))
#     generator.add(BatchNormalization())
    
    generator.add(Dense(11100))
#     generator.add(Activation('relu'))
#     generator.add(Dense(10000))
    generator.add(BatchNormalization())
    generator.add(Activation('relu'))
    
#     generator.add(Dense(10000, input_dim=gen_input_size))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(7100))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(encoding_dim))
    generator.add(Dense(data_train_cohort.shape[1], kernel_initializer = my_innit, bias_initializer = my_binnit, trainable=False, activation='sigmoid'))
    
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return generator

def discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(11100, input_dim=data_train_cohort.shape[1]))
    discriminator.add(BatchNormalization())
    discriminator.add(LeakyReLU(0.2))
    
#     discriminator.add(Dense(50))
#     discriminator.add(BatchNormalization())
#     discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss = 'binary_crossentropy', optimizer=optimizer)
    
    return discriminator

discriminator = discriminator()
generator = generator()

discriminator.trainable = False

gan_input = Input(shape=(gen_input_size,))
generated_patient = generator(gan_input)
gan_output = discriminator(generated_patient)

gan = Model(gan_input, gan_output)

gan.compile(loss='binary_crossentropy', optimizer=optimizer)
start = time.time()
for epoch in range(epochs):
    for steps in range(steps_per_epoch):
        noise = sample_z(batch_size, gen_input_size)
        fake_x = generator.predict(noise)
        real_x = data_train_cohort.sample(n=batch_size)
        x = np.concatenate((real_x, fake_x))
        disc_y = np.zeros(2*batch_size)
        disc_y[:batch_size] = 0.9
        d_loss = discriminator.train_on_batch(x, disc_y)
        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)
        
    if epoch%1000 == 0:
        print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(epoch,d_loss,g_loss))
        print((time.time()-start)/60)

NameError: name 'config' is not defined

In [7]:
a = generator.predict(sample_z(1, gen_input_size)) #Generate cohort of patients of size cohort size from generator
num_patients = 10000 #Number of patients we want to generate - note this should be divisible by cohort_size
for i in range((int(num_patients/cohort_size))):
    a = np.concatenate((a, generator.predict(sample_z(1, gen_input_size))), axis=0)
print(a.shape)
fake_data = np.reshape(a, (int(a.shape[1]/data_cat.shape[1]*a.shape[0]), data_cat.shape[1])) #Reshape generated patients array so you have 1 patient per row
fake_data = pd.DataFrame(b, columns=data_cat.columns.values)
target_ls = []
target_df = fake_data[data_cat.columns.values[11]] #This is our target variable from the fake dataset - whether the patient was hospitalised
for i in range(fake_data.shape[0]):
    if target_df[i] ==1:
        target_ls.append(1)
    else:
        target_ls.append(0)
print(sum(target_ls))        
fake_data = fake_data.drop([data_cat.columns.values[11], data_cat.columns.values[12]], axis=1) #Remove the target variable to give us our fake training dataset
test_label = data_test[data_cat.columns.values[11]] #Creating the test label from our test dataset which we created at the start
test_data = data_test.drop([data_cat.columns.values[11], data_cat.columns.values[12]], axis=1) #creating test dataset
real_train_data = data_train #Training dataset made up of our real data 
real_train_label = real_train_data[data_cat.columns.values[11]] #Labet data from real dataset
real_train_data = real_train_data.drop([data_cat.columns.values[11], data_cat.columns.values[12]], axis=1)
# variable_train, test_data, label_train, test_label = train_test_split(test_data, test_label, test_size=0.20)

#Develop logistic regression models using real and fake data - Objective is to build 
# a model using fake data which is at least equal to the model using real data
logreg_gan = LogisticRegression()
logreg_real = LogisticRegression()
logreg_gan.fit(b, target_ls)
logreg_real.fit(real_train_data, real_train_label)
accuracy_gan = round(logreg_gan.score(test_data, test_label)*100, 2)
accuracy_real = round(logreg_real.score(test_data, test_label)*100, 2)

print(accuracy_gan)
print(accuracy_real)
print(confusion_matrix(test_label, logreg_gan.predict(test_data)))
# print(label_test.describe())
# print(a)

NameError: name 'generator' is not defined